In [4]:
import gym
%matplotlib inline
import numpy as np
import tensorflow as tf

In [2]:
env = gym.make('CartPole-v0')

# 1 model

In [3]:
n_input = 4;n_hidden1 = 4;n_output = 1

In [5]:
x1 = tf.placeholder(shape=[None,n_input],name='input_x',dtype=tf.float32)

In [6]:
w1 = tf.Variable(tf.random_normal([n_input,n_hidden1]),name='w1');
b1 = tf.Variable(tf.zeros(shape=[n_input]),name='b1',dtype=tf.float32)

In [7]:
h1 = tf.nn.elu(tf.matmul(x1,w1)+b1)

In [8]:
w2 = tf.Variable(tf.random_normal([n_hidden1,n_output]),name='w2',dtype=tf.float32)
b2 = tf.Variable(tf.zeros(shape=[n_output]),name='b2',dtype=tf.float32)

In [11]:
logits = tf.matmul(h1,w2)+b2
h2 = tf.nn.sigmoid(logits)

In [12]:
##select action accoding output
p_left_or_right = tf.concat([h2,1-h2],1)

In [13]:
#概率采样
action = tf.multinomial(tf.math.log(p_left_or_right),num_samples=1)

Instructions for updating:
Use `tf.random.categorical` instead.


In [15]:
##target probability !! local target 
y = 1- tf.to_float(action)

In [17]:
#loss  !! local loss
cross_entroy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=logits)

In [18]:
##optimizer  !!local optimizer for get gradients
optimizer = tf.train.AdadeltaOptimizer(learning_rate=1e-2)

In [19]:
grads_and_vars = optimizer.compute_gradients(cross_entroy)

In [20]:
gradients = [grad for grad,var in grads_and_vars]

In [21]:
grads_and_vars

[(<tf.Tensor 'gradients/MatMul_grad/tuple/control_dependency_1:0' shape=(4, 4) dtype=float32>,
  <tf.Variable 'w1:0' shape=(4, 4) dtype=float32_ref>),
 (<tf.Tensor 'gradients/add_grad/tuple/control_dependency_1:0' shape=(4,) dtype=float32>,
  <tf.Variable 'b1:0' shape=(4,) dtype=float32_ref>),
 (<tf.Tensor 'gradients/MatMul_3_grad/tuple/control_dependency_1:0' shape=(4, 1) dtype=float32>,
  <tf.Variable 'w2:0' shape=(4, 1) dtype=float32_ref>),
 (<tf.Tensor 'gradients/add_3_grad/tuple/control_dependency_1:0' shape=(1,) dtype=float32>,
  <tf.Variable 'b2:0' shape=(1,) dtype=float32_ref>)]

In [23]:
##此处只为plicy更新梯度时使用，实现的很复杂。。
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    ##对每个参数 分别进行梯度处理
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape()) 
    gradient_placeholders.append(gradient_placeholder) ## 记录梯度
    grads_and_vars_feed.append((gradient_placeholder, variable)) ## 外部处理梯度后，将梯度回填
    training_op = optimizer.apply_gradients(grads_and_vars_feed) ##更新特参数的梯度

In [24]:
##init
init = tf.global_variables_initializer()

In [25]:
saver = tf.train.Saver()

# 2 reward 处理

In [26]:
def discount_rewards(rewards,discount_rate):
    '''
    计算单轮内部，每个action的收益
    '''
    discounted_reward = np.empty_like(rewards)
    comulative_reward = 0
    for index in range(len(rewards)-1,-1,-1):
        #print(index)
        comulative_reward = discount_rate*comulative_reward + rewards[index]
        discounted_reward[index] = comulative_reward
    return discounted_reward

In [27]:
def discount_and_normalize_rewards(all_rewards,discount_rate):
    '''
    计算单轮内部，每个action的收益并且归一化
    '''
    all_discounted_rewards = [discount_rewards(rewards,discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    #print(flat_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

# 3 train 

In [29]:
n_iterations = 250 # number of training iterations
n_max_steps = 1000 # max steps per episode
n_games_per_update = 10 # train the policy every 10 episodes 
save_iterations = 10 # save the model every 10 training iterations 
discount_rate = 0.95

## policy运行阶段，只累计梯度； 这里的梯度只是根据targt probability计算出来的； 而实际需要调整的梯度，是此时的梯度乘以gain的方向。
  ##:gain变大，或者gain变小，通过gain normalize来提供梯度更新的方向。

In [ ]:
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):## 迭代10次；
        all_rewards = [] ##每轮的收益， （10，（1000）） 10个1000维的向量
        all_gradients = [] ##每个step的梯度 （10，（1000））10个1000维的向量
        for game in range(n_games_per_update):##每10轮更新一次，每轮1000步
            current_rewards = []   ##当前轮每个动作的收益 1000个
            current_gradients = []  ##当前轮每个梯度 1000个
            ##每轮重启环境
            obs = env.reset()
            for step in range(n_max_steps):
                action_val,gradients_val = sess.run([action,gradients],feed_dict={x1:obs.reshape(1,n_input)})
                ##获取action产生的reward
                obs,reward,done,info = env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
        ##每10轮结束时，更新policy的参数
        
        all_rewards_norm = discount_and_normalize_rewards(all_rewards,0.95) 
        feed_dict = {}
        ##计算每个变量的梯度
        for var_index, grad_placeholder in enumerate(gradient_placeholders):
            # multiply the gradients by the action scores, and compute the mean , 
            #归一化后的rewards 给梯度一个合适的方向，
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index] \
                                      for game_index, rewards in enumerate(all_rewards_norm) \
                                      for step, reward in enumerate(rewards)],axis=0)
            feed_dict[grad_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict) 
        print([sum(gain) for gain in all_rewards])
        if iteration % save_iterations == 0:
            saver.save(sess, "./my_policy_net_pg.ckpt")
            

[13.0, 22.0, 54.0, 19.0, 16.0, 35.0, 26.0, 17.0, 20.0, 36.0]
[21.0, 13.0, 18.0, 35.0, 18.0, 26.0, 70.0, 20.0, 36.0, 14.0]
[60.0, 19.0, 72.0, 28.0, 41.0, 19.0, 21.0, 79.0, 44.0, 18.0]
[14.0, 12.0, 30.0, 14.0, 36.0, 14.0, 27.0, 36.0, 24.0, 12.0]
[15.0, 85.0, 23.0, 23.0, 38.0, 25.0, 48.0, 85.0, 16.0, 11.0]
[57.0, 27.0, 11.0, 9.0, 29.0, 42.0, 21.0, 25.0, 17.0, 20.0]
[41.0, 12.0, 45.0, 33.0, 39.0, 82.0, 15.0, 18.0, 24.0, 42.0]
[77.0, 36.0, 17.0, 11.0, 13.0, 18.0, 70.0, 43.0, 21.0, 56.0]
[35.0, 21.0, 23.0, 15.0, 42.0, 24.0, 14.0, 51.0, 17.0, 21.0]
[19.0, 44.0, 34.0, 46.0, 15.0, 10.0, 30.0, 41.0, 37.0, 13.0]
[20.0, 11.0, 13.0, 36.0, 32.0, 15.0, 26.0, 80.0, 14.0, 21.0]
[50.0, 15.0, 14.0, 40.0, 19.0, 10.0, 28.0, 12.0, 19.0, 16.0]
[21.0, 12.0, 18.0, 27.0, 27.0, 21.0, 44.0, 15.0, 84.0, 33.0]
[22.0, 37.0, 18.0, 17.0, 21.0, 43.0, 27.0, 25.0, 68.0, 20.0]
[68.0, 15.0, 91.0, 18.0, 57.0, 30.0, 50.0, 39.0, 47.0, 18.0]
[17.0, 69.0, 53.0, 14.0, 12.0, 12.0, 28.0, 14.0, 52.0, 33.0]
[24.0, 15.0, 21.0, 14.0, 

[62.0, 28.0, 20.0, 13.0, 38.0, 59.0, 24.0, 12.0, 56.0, 15.0]
[98.0, 13.0, 32.0, 31.0, 74.0, 47.0, 26.0, 46.0, 12.0, 18.0]
[12.0, 12.0, 18.0, 56.0, 22.0, 30.0, 24.0, 24.0, 33.0, 19.0]
[17.0, 53.0, 25.0, 31.0, 25.0, 49.0, 35.0, 24.0, 23.0, 19.0]
[23.0, 11.0, 13.0, 56.0, 39.0, 22.0, 12.0, 12.0, 27.0, 29.0]
[69.0, 12.0, 16.0, 26.0, 24.0, 59.0, 19.0, 25.0, 23.0, 71.0]
[22.0, 53.0, 33.0, 23.0, 20.0, 67.0, 37.0, 54.0, 12.0, 31.0]
[14.0, 16.0, 15.0, 12.0, 35.0, 14.0, 110.0, 15.0, 13.0, 69.0]
[19.0, 47.0, 18.0, 43.0, 15.0, 26.0, 18.0, 26.0, 20.0, 32.0]
[35.0, 17.0, 11.0, 31.0, 70.0, 18.0, 36.0, 30.0, 22.0, 15.0]
[22.0, 19.0, 33.0, 32.0, 15.0, 27.0, 17.0, 18.0, 13.0, 32.0]
[45.0, 58.0, 13.0, 19.0, 39.0, 22.0, 45.0, 20.0, 19.0, 67.0]
[26.0, 26.0, 16.0, 27.0, 30.0, 22.0, 13.0, 39.0, 33.0, 24.0]
[27.0, 79.0, 19.0, 17.0, 40.0, 26.0, 17.0, 28.0, 59.0, 8.0]
[41.0, 32.0, 25.0, 51.0, 16.0, 19.0, 20.0, 55.0, 49.0, 34.0]
[25.0, 16.0, 70.0, 30.0, 42.0, 13.0, 38.0, 30.0, 29.0, 21.0]
[28.0, 17.0, 20.0, 13.0,